In [ ]:
import pandas as pd
import numpy  as np

import sklearn
from sklearn import linear_model
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler

from matplotlib import pyplot as plt
from matplotlib import cm
import seaborn as sns

from sklearn import tree, svm, neighbors
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, StackingRegressor
from sklearn.linear_model import LinearRegression, Ridge

### Первичный анализ датасета

**Предметная область** - доход и размеры домашних хозяйств Калифорнии по данным до 1990г.
Признаки в таблице:
1) Median House Value: Медианная цена дома в квартале[$]
2) Median Income: Медианный доход на семью в квартале[10тыс.$] (TARGET)
3) Median Age: Медианный возраст дома в квартале; меньше = новее [лет]
4) Total Rooms: Общее количество комнат в квартале
5) Total Bedrooms: Общее количество спален в квартале
6) Population: Общее количество людей, проживающих в квартале
7) Households: Общее количество домохозяйств, групп людей, проживающих в жилом помещении, в квартале
8) Latitude: показатель того, насколько далеко на север находится дом; чем выше значение, тем дальше на север [°]
9) Longitude: показатель того, насколько далеко на запад находится дом; чем выше значение, тем дальше на запад [°]
10) Distance to coast: расстояние до ближайшей точки побережья [м]
11) Distance to Los Angeles: расстояние до центра Лос-Анджелеса [м]
12) Distance to San Diego: расстояние до центра Сан-Диего [м]
13) Distance to San Jose: расстояние до центра Сан-Хосе [м]
14) Distance to San Francisco: расстояние до центра Сан-Франциско [м]
15) Distance to the ocean (Расстояние до океана)

Структура csv-файла:
- Ячейки разделены ";"
- Дробная часть помечена "."
- Нечисловой признак - ocean_proximity
- Дат не представлено
- Null помечены "?$?" (единственная запись)

In [ ]:
dataset_file = "datasets/dataset_prepared.csv"
df = pd.read_csv(
    dataset_file,
    sep=';',
    decimal='.',
    header=0,
    na_values=['?$?']
)

df.head()

Выясним размеры датасета

In [ ]:
num_rows, num_cols = df.shape
print(f"Размеры набора данных: {num_rows} строк и {num_cols} столбцов\n")
df.info()

---

### Выбор целевого значения и признаков для анализа данных

Целевой признак - Median Income: Медианный доход на семью в квартале[10тыс.$].

Построим матрицу корреляции (коэфф. корреляции Пирсона) для возможных комбинай пар.

In [ ]:
target=['Median_Income']
corr_coeffs = df.corr(method='pearson')
corr_coeffs.style.background_gradient(cmap='coolwarm', axis=None)
corr_coeffs[target[0]].abs().sort_values(ascending=False)

In [ ]:
mask = np.zeros_like(corr_coeffs, dtype=bool)
mask[np.triu_indices_from(mask)] = True

fig, ax = plt.subplots(figsize=(12,12))
sns.set_theme(font_scale=0.8)

sns.heatmap(
            data=corr_coeffs,
            vmin=-1, vmax=1, center=0,
            annot=True,
            cmap = 'RdBu_r',
            mask=mask,
            square=True,
            linewidths=1.0,
            cbar_kws={"shrink": .7}
           )
plt.title(u'Матрица корреляции признаков');

В качестве независимых переменных выберем признаки с высоким абс. значением коэфф. корреляции, но при этом как можно более не связанные между собой. Кандидаты:
- *Median_House_Value* - Медианная цена дома в квартале [$]
- *Distance_to_coast*- расстояние до ближайшей точки побережья [м] (или *OcPrx_INLAND*)
- *Tot_Rooms* - Общее количество комнат в квартале
- *Median_Age* - Медианный возраст дома в квартале; меньше = новее [лет]

In [ ]:
features = ['Median_House_Value', 'Distance_to_coast', 'Tot_Rooms',  'Median_Age', 'Population']

Получим следующие распределения значений

In [ ]:
# dataset_filtered[features].hist() #Упрощенный вывод графиков

plt.figure(figsize=(16, 5))
plot_number = 0

for feature_name in (features+target):
    plot_number += 1
    plt.subplot(1, len(features+target), plot_number)
    plt.hist(df[feature_name])

    plt.title(feature_name)
    plt.xlabel(u'Значения')
    plt.ylabel(u'Частота')
    print (feature_name,
           df[feature_name].min(),
           df[feature_name].max())

Уберем все прочие признаки

In [ ]:
df = df[target +features]
df.head()

### Объявление функций

In [ ]:
def plot_difference(y_test, y_pred) -> None:
    '''
    Функция построения графиков
    :param y_test: - проверочные значения целевой переменной
    :param y_pred: - вычисленные значения целевой переменной
    '''
    plt.figure(figsize=(12,6))
    
    plt.subplot(121)
    plt.scatter(y_test, y_pred,  alpha=0.1, color = "#17becf")
    plt.axline((0, 0), slope=1, color='black', linestyle='--', linewidth=3, alpha=0.7,)
    range_test, range_pred = np.max(y_test)- np.min(y_test) , np.max(y_pred)- np.min(y_pred)
    if range_test/range_pred <4 and range_test/range_pred > 0.25:
        plt.gca().set_aspect('equal')
        axmin, axmax = np.min([y_test, y_pred]), np.max([y_test, y_pred])
        plt.xlim([axmin, axmax]); plt.ylim([axmin, axmax]);
    plt.title('Диаграмма рассеяния вычисленных значений');
    plt.xlabel('Проверочное Y')
    plt.ylabel('Вычисленное Y')
    plt.grid(True)

    plt.subplot(122) # 1 row, 2 column, 2 index on grid
    plt.scatter(y_test, (y_test - y_pred)**2,  alpha=0.1, color = "#17becf")
    plt.title('Диаграмма рассеяния квадрата абсолютной ошибки')
    plt.xlabel('Проверочное Y')
    plt.ylabel('Квадрат абсолютной ошибки')
    plt.grid(True)

In [ ]:
def stats(y_test, y_pred):
    '''
    Вычисление и вывод метрик: MAE, RMSE, R2. Используются функции из библиотеки sklearn
    На основе сравнения проверочных и вычисленных.
    :param y_test: - проверочные значения целевой переменной
    :param y_pred: - вычисленные значения целевой переменной
    '''
    mae  = metrics.mean_absolute_error(y_test, y_pred)
    mse  = metrics.mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2   = metrics.r2_score(y_test, y_pred)
    return {"MAE":mae, "MSE":mse, "RMSE":rmse, "R2":r2}

In [ ]:
def error_distribution(y_test_values, y_pred, max_prob=0.35):
    plt.figure(figsize=(6,4))
    sns.histplot( data = (y_test_values - y_pred),
                  color="red",
                  kde=True, # оценка плотности в виде кривой
                  stat="density",# density: общая площадь равна 1
                )
    # Осевая линия
    plt.plot(
        [0, 0],
        [0, max_prob], '--', lw=2, c='r')

    plt.ylabel(u'Плотность')
    plt.xlabel(u'Значение ошибки')
    plt.title(u'Плотность распределения и гистограмма ошибок');
    plt.show()

In [ ]:
def plot_params(x_vals, y_vals, name, xlabel='Параметр', ylabel='RMSE'):
    plt.plot(x_vals, y_vals, marker='o')
    plt.title(name)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.show()

In [ ]:
def plot_params_multiple(x_vals, y_vals_list, names, title, xlabel='Параметр', ylabel='RMSE'):
    plt.figure(figsize=(8,5))
    for y_vals, name in zip(y_vals_list, names):
        plt.plot(x_vals, y_vals, marker='o', label=name)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.show()

### Разделение выборки

Нужно взять тренировочную и проверучную часть с уже вычисленными значениями таргета. Пусть доля тестов - 20%

In [ ]:
from sklearn.model_selection import train_test_split

test_size = 0.2
rnd_state = 8 # for reproducibility

x_train, x_test, y_train, y_test =  train_test_split(
    df[features],
    df[target[0]],
    test_size = test_size,
    random_state=rnd_state,
    shuffle=True
)

print(x_train.head())
print(x_test.head())

In [ ]:
print ("Кол-во элементов: \n  x_train: {}, y_train {} \n  x_test:  {}, y_test  {} \n  total x: {}, total y {} ".format  (
    len(x_train), len(y_train),
    len(x_test),  len(y_test),
    len(x_train)+len(x_test), len(y_train)+len(y_test),
))

### Линейная регрессия

In [ ]:
print("Признаки:", features)
model_all = linear_model.LinearRegression()
model_all.fit(
    x_train[features],
    y_train
)

In [ ]:
y_all = model_all.predict(x_test)

In [ ]:
error_distribution(y_test, y_all)

In [ ]:
plot_difference(
    y_test = y_test,
    y_pred = y_all
)

In [ ]:
stats(
    y_test=y_test,
    y_pred=y_all
)

### Бэггинг

In [ ]:
print("Признаки:", features)
model_bag = BaggingRegressor()
model_bag.fit(
    x_train,
    y_train
)

In [ ]:
y_bag = model_bag.predict(x_test)

In [ ]:
error_distribution(y_test, y_bag)

In [ ]:
plot_difference(
    y_test = y_test,
    y_pred = y_bag
)

In [ ]:
stats(
    y_test=y_test,
    y_pred=y_bag
)

#### Подбор параметров

In [ ]:
n_estimators_list = [10, 50, 100, 200, 500, 1000]
bagging_rmse = []
bagging_r2 = []
for n in n_estimators_list:
    model = BaggingRegressor(n_estimators=n, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    st=stats(y_test, y_pred)
    bagging_rmse.append(st['RMSE'])
    bagging_r2.append(st['R2'])

In [ ]:
plot_params(
    x_vals=n_estimators_list,
    y_vals=bagging_rmse,
    name='Bagging Regressor: Зависимость RMSE от числа оценивателей',
    xlabel='Число оценивателей',
    ylabel='RMSE'
)

In [ ]:
plot_params(
    x_vals=n_estimators_list,
    y_vals=bagging_r2,
    name='Bagging Regressor: Зависимость R2 от числа оценивателей',
    xlabel='Число оценивателей',
    ylabel='R2'
)

### Случайный лес

In [ ]:
model_rf = RandomForestRegressor()
model_rf.fit(
    x_train,
    y_train
)

In [ ]:
y_rf = model_rf.predict(x_test)

In [ ]:
error_distribution(y_test, y_rf)

In [ ]:
plot_difference(
    y_test = y_test,
    y_pred = y_rf
)

In [ ]:
stats(
    y_test=y_test,
    y_pred=y_rf
)

#### Подбор параметров

In [ ]:
max_depth_list = [3, 5, 10, None]
rf_rmse = []
rf_r2 = []
for depth in max_depth_list:
    model = RandomForestRegressor(max_depth=depth, n_estimators=100, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    st = stats(y_test, y_pred)
    rf_rmse.append(st['RMSE'])
    rf_r2.append(st['R2'])
    

In [ ]:
plot_params(
    x_vals=['3', '5', '10', 'None'],
    y_vals=rf_rmse,
    name='Random Forest Regressor: Зависимость RMSE от максимальной глубины',
    xlabel='Максимальная глубина',
    ylabel='RMSE'
)

In [ ]:
plot_params(
    x_vals=['3', '5', '10', 'None'],
    y_vals=rf_r2,
    name='Random Forest Regressor: Зависимость R2 от максимальной глубины',
    xlabel='Максимальная глубина',
    ylabel='R2'
)

### Бустинг

In [ ]:
model_gbr = GradientBoostingRegressor()
model_gbr.fit(
    x_train,
    y_train
)

In [ ]:
y_gbr = model_gbr.predict(x_test)

In [ ]:
error_distribution(y_test, y_gbr)

In [ ]:
plot_difference(
    y_test = y_test,
    y_pred = y_gbr
)

In [ ]:
stats(
    y_test=y_test,
    y_pred=y_gbr
)

In [ ]:
# # 4. StackingRegressor: final_estimator
# base_models = [
#     ('ridge', linear_model.Ridge()),
#     ('tree', tree.DecisionTreeRegressor(max_depth=5)),
#     ('svm', svm.SVR())
# ]
# final_models = [linear_model.LinearRegression(), tree.DecisionTreeRegressor(max_depth=5), ensemble.GradientBoostingRegressor(n_estimators=50)]
# st_names = ['Linear', 'Tree', 'GBR']
# stacking_rmse = []
# for meta in final_models:
#     model = StackingRegressor(estimators=base_models, final_estimator=meta, passthrough=False)
#     model.fit(x_train, y_train)
#     y_pred = model.predict(x_test)
#     stacking_rmse.append(stats(y_test, y_pred)['RMSE'])
# results['StackingRegressor'] = (st_names, stacking_rmse)


#### Подбор параметров

In [ ]:
learning_rates = [0.001, 0.01, 0.1, 0.3]
n_estimators_list_boosting = [10, 50, 100, 200, 500, 1000]
gbr_rmse = {lr: [] for lr in learning_rates}
gbr_r2   = {lr: [] for lr in learning_rates}
for lr in learning_rates:
    for n in n_estimators_list_boosting:
        model = GradientBoostingRegressor(learning_rate=lr, n_estimators=n, random_state=42)
        model.fit(x_train, y_train)
        y_pred = model.predict(x_test)
        st = stats(y_test, y_pred)
        gbr_rmse[lr].append(st['RMSE'])
        gbr_r2[lr].append(st['R2'])

In [ ]:
plot_params_multiple(
    x_vals=n_estimators_list_boosting,
    y_vals_list=[gbr_rmse[lr] for lr in learning_rates],
    names=[f'LR={lr}' for lr in learning_rates],
    title='Gradient Boosting Regressor: Зависимость RMSE от числа оценивателей при разных скоростях обучения',
    xlabel='Число оценивателей',
    ylabel='RMSE'
)

In [ ]:
plot_params_multiple(
    x_vals=n_estimators_list_boosting,
    y_vals_list=[gbr_r2[lr] for lr in learning_rates],
    names=[f'LR={lr}' for lr in learning_rates],
    title='Gradient Boosting Regressor: Зависимость R2 от числа оценивателей при разных скоростях обучения',
    xlabel='Число оценивателей',
    ylabel='R2'
)

In [ ]:
learning_rate_fun = 0.001
n_estimators_list_boosting_fun = [10, 50, 100, 200, 500, 1000, 2000, 5000, 10000, 20000]
gbr_rmse_fun = []
gbr_r2_fun = []
for n in n_estimators_list_boosting_fun:
    model = GradientBoostingRegressor(learning_rate=learning_rate_fun, n_estimators=n, random_state=42)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    st = stats(y_test, y_pred)
    gbr_rmse_fun.append(st['RMSE'])
    gbr_r2_fun.append(st['R2'])

In [ ]:
plot_params(
    x_vals=n_estimators_list_boosting_fun,
    y_vals=gbr_rmse_fun,
    name=f'Gradient Boosting Regressor: Зависимость RMSE от числа оценивателей при learning_rate={learning_rate_fun}',
    xlabel='Число оценивателей',
    ylabel='RMSE'
)

In [ ]:
plot_params(
    x_vals=n_estimators_list_boosting_fun,
    y_vals=gbr_r2_fun,
    name=f'Gradient Boosting Regressor: Зависимость R2 от числа оценивателей при learning_rate={learning_rate_fun}',
    xlabel='Число оценивателей',
    ylabel='R2'
)

# Итоги

## Метрики моделей

In [ ]:
from IPython.display import display, Markdown

In [ ]:
def display_stats(model_description, feature_str, model, y_test, y_pred):
    mapped_stats = stats(y_test, y_pred)
    mae  = mapped_stats["MAE"]
    mse  = mapped_stats["MSE"]
    rmse = mapped_stats["RMSE"]
    r2   = mapped_stats["R2"]
    
    stats_str=f"""
    {model_description}. Признаки: '{feature_str}'.
    MAE : {mae:>9,.3f} (средняя абсолютная ошибка)
    MSE : {mse:>9,.6f} (среднеквадратичная ошибка)
    RMSE: {rmse:>9,.6f} (кв. корень из среднеквадратичной ошибки)
    R2  : {r2:>9,.3f} (коэфф. детерминации)
    """
    
    display(Markdown(stats_str))

display_stats('Линейная многомерная от 5х признаков', features, model_all, y_test, y_all)
display_stats('Бэггинг от 5х признаков', features, model_bag, y_test, y_bag)
display_stats('Случайный лес от 5х признаков', features, model_rf, y_test, y_rf)
display_stats('Бэггинг от 5х признаков', features, model_gbr, y_test, y_gbr)